# HW4 KAMENEV VLADIMIR

### Доступные колонки
##### date_time - время показа рекламы
##### oaid_hash - хэш юзера
##### zone_id - id зоны, где зона - место на сайте для размещения рекламы
##### banner_id - id баннера, где баннер - сама реклама
##### os_id - id операционной системы
##### country_id - id страны
##### banner_id0 - нулевой баннер в “стакане” баннеров
##### banner_id1 - перый баннер в “стакане” баннеров
##### rate0 - стоимость 1 клика установленная рекламодателем для banner_id0
##### rate1 - стоимость 1 клика установленная рекламодателем для banner_id1
##### g0 - стандартное отклонение предикта с banner_id0
##### g1 - стандартное отклонение предикта с banner_id1
##### coeff_sum0 - сумма коэффициентов для banner_id0
##### coeff_sum1 - сумма коэффициентов для banner_id1
##### impressions - был ли показ
##### clicks - был ли клик

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import cross_validate

/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('../..//data.csv')
# data = data.sample(frac=0.1) #
data['date_time'] = data['date_time'].apply(lambda x: datetime.strptime(x.split('.')[0],'%Y-%m-%d %H:%M:%S'))# для удобной работы со временем
data = data.sort_values(by=['date_time']) 

In [ ]:
used_cols = ['date_time','oaid_hash','zone_id','banner_id','os_id','country_id','banner_id0','banner_id1','rate0','rate1','g0','g1',
'coeff_sum0','coeff_sum1','clicks']

In [4]:
data = data[data['date_time'].dt.date != pd.Timestamp('2021-09-01').date()]

In [7]:
used_data = data[used_cols]

In [10]:
used_data = used_data.drop_duplicates()
used_data['month'] = used_data['date_time'].dt.month
used_data['day'] = used_data['date_time'].dt.day
used_data['hour'] = used_data['date_time'].dt.hour
used_data['minute'] = used_data['date_time'].dt.minute
used_data['second'] = used_data['date_time'].dt.second
used_data['is_day'] = used_data['hour'].apply(lambda x: 1 if int(x)>=12 and int(x) <=24 else 0) # Был ли показ днем
used_data['is_night'] = used_data['is_day'].apply(lambda x: 1 if x==0 else 0) # Был ли показ ночью
used_data['is_weekend'] = used_data['date_time'].apply(lambda x: 1 if x.weekday() > 4 else 0) # Был ли показ на выходных

In [11]:
used_data

,date_time,oaid_hash,zone_id,banner_id,os_id,country_id,banner_id0,banner_id1,rate0,rate1,...,impressions,clicks,month,day,hour,minute,second,is_day,is_night,is_weekend
5041415,2021-09-26 00:00:00,1834033519797437404,41,29,3,0,29,6,0.0020,0.002,...,1,0,9,26,0,0,0,0,1,1
1442602,2021-09-26 00:00:00,7416450538971744701,1,188,2,15,188,11754925,0.0080,0.330,...,1,0,9,26,0,0,0,0,1,1
7232498,2021-09-26 00:00:00,1832228443297591417,17,52,2,5,52,41,0.0080,0.004,...,1,0,9,26,0,0,0,0,1,1
14938691,2021-09-26 00:00:00,4180077124914749282,47,73,4,13,73,1040,0.0080,0.008,...,1,0,9,26,0,0,0,0,1,1
11536774,2021-09-26 00:00:00,1459689388363839798,48,266,0,1,266,124,0.0050,0.003,...,1,0,9,26,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10139863,2021-10-02 23:59:59,1030486855470422958,24,180,2,6,180,403,0.0080,0.008,...,1,0,10,2,23,59,59,1,0,1
5597133,2021-10-02 23:59:59,5392241310084555538,73,92,1,0,92,12584620,0.0594,1.500,...,1,0,10,2,23,59,59,1,0,1
8660907,2021-10-02 23:59:59,5569517219693927594,17,1235,4,0,1235,1240,0.0670,0.067,...,1,0,10,2,23,59,59,1,0,1
2991997,2021-10-02 23:59:59,4888551521096100763,1,2,0,0,2,21,0.0140,0.014,...,1,0,10,2,23,59,59,1,0,1


In [19]:
feats = ['zone_id','banner_id','os_id','country_id','month','day','hour',
        'is_day','is_night','is_weekend']
feats_to_transform = ['zone_id','banner_id','os_id','country_id']
target = 'clicks'

In [20]:
end_last_day = used_data['date_time'].iloc[-1]
start_last_day = end_last_day - pd.to_timedelta(1, unit='d')

In [21]:
X_train = used_data[(used_data['date_time']<start_last_day)][feats]
Y_train = used_data[(used_data['date_time']<start_last_day)][target]

In [22]:
# X_test = used_data[(data['date_time']>start_last_day) & (data['date_time']<=end_last_day)][feats]
# Y_test = used_data[(data['date_time']>start_last_day) & (data['date_time']<=end_last_day)][target]

In [23]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
X_train = ohe.fit_transform(X_train)
# X_val = ohe.transform(X_val)

In [24]:
feature_selector = SelectKBest(chi2, k=1000)

In [25]:
X_train = feature_selector.fit_transform(X_train, Y_train)

In [ ]:
X_test = ohe.transform(X_test)
X_test = feature_selector.transform(X_test)


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV

In [30]:
def create_model():
    return LogisticRegression(solver='liblinear', penalty='l2')

In [31]:
def cv(X_train,Y_train): ### Подбираем параметр C
    model = create_model()
    param_search = {'C' : [0.001, 0.01, 0.1, 1.]}

    splits = TimeSeriesSplit(n_splits=3)
    clf = GridSearchCV(estimator=model, cv=splits,
                            param_grid=param_search, verbose=10, scoring=['roc_auc', 'neg_log_loss'], refit="neg_log_loss")
    clf.fit(X_train, Y_train)
    
    return clf

In [32]:
%%time
best_model = cv(X_train,Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3; 1/4] START C=0.001.....................................................
[CV 1/3; 1/4] END C=0.001; neg_log_loss: (test=-0.086) roc_auc: (test=0.668) total time=   8.3s
[CV 2/3; 1/4] START C=0.001.....................................................
[CV 2/3; 1/4] END C=0.001; neg_log_loss: (test=-0.112) roc_auc: (test=0.670) total time=  16.9s
[CV 3/3; 1/4] START C=0.001.....................................................
[CV 3/3; 1/4] END C=0.001; neg_log_loss: (test=-0.153) roc_auc: (test=0.721) total time=  29.0s
[CV 1/3; 2/4] START C=0.01......................................................
[CV 1/3; 2/4] END C=0.01; neg_log_loss: (test=-0.084) roc_auc: (test=0.694) total time=  14.4s
[CV 2/3; 2/4] START C=0.01......................................................
[CV 2/3; 2/4] END C=0.01; neg_log_loss: (test=-0.110) roc_auc: (test=0.702) total time=  25.8s
[CV 3/3; 2/4] START C=0.01...............................

In [33]:
best_model

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0]}, refit='neg_log_loss',
             scoring=['roc_auc', 'neg_log_loss'], verbose=10)